In [79]:
# packages
import os
import numpy as np
import pandas as pd
import pickle
import ray 
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import IFrame
from joblib import Parallel, delayed
from src.model_2state import solve_prep, solve_pre_jump_2state
from src.simulation_2d import simulation_2d, simulate_logkapital
from src.plots import plot_2S_ey1y2, plot_1S_vs_2S_ems, plot_1S_vs_2S_SCC, plot_2S_ey1y2_multi_λ, plot_1S_vs_2S_ems_multi_λ, plot_1S_vs_2S_SCC_multi_λ 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams["figure.edgecolor"] = "w"
mpl.rcParams['figure.facecolor'] = "w"
mpl.rcParams["savefig.dpi"] = 300

# Appendix C: when $Y$ has two states:

Process:

$$
dY_t^1  = Y_t^2 dt
$$

$$
dY_t^2 =   - \lambda Y_t^2 dt + \lambda \theta \mathcal{E} dt
$$


## Step I: post jump HJB:

The post jump HJB for $\phi_m(y_1, y_2)$, $m = 1, \dots, M$:

$$
\begin{aligned}
0 = \max_{\mathcal{E}} \min_{\omega_\ell } & - \delta \phi_m(y_1, y_2) + \eta log(\mathcal{E}) \\
& + \frac{\partial \phi_m}{\partial y_1} y_2 + \frac{\partial \phi_m}{\partial y_2} \lambda (- y_2 + \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} \left(\gamma_1 + \gamma_2 y_1 + \gamma_3 (y_1 - \bar y)\mathbb{I}\{y_1>\bar y\} \right) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)
\end{aligned}
$$

First order condition for $\omega_\ell$, $\ell = 1, \dots, L$:

$$
    \omega_\ell \propto \pi_\ell^a \exp\left( -\frac{1}{\xi_a} \frac{\partial \phi_m}{\partial y_2}\lambda \theta_\ell \mathcal{E} \right), for \ell = 1, \dots, L
$$

and the first order condition for emission is:
$$
\mathcal{E} = - \cfrac{\eta}{\frac{\partial \phi_m }{\partial y_2} \lambda \sum_{\ell=1}^{L} \omega_\ell \theta_\ell}
$$


## Step II: pre jump HJB:

Given post jump value functions $\phi_m(y_1, y_2)$, $m = 1, 2, M$, solve the following HJB for pre-jump value function $\Phi(y_1, y_2)$:

$$
\begin{aligned}
0 = \max_{\mathcal{E}}\min_{\omega_\ell }  & - \delta \Phi(y_1, y_2) +  \eta log(\mathcal{E}) \\
& + \frac{\partial \Phi}{\partial y_1} y_2 + \frac{\partial \Phi}{\partial y_2} \lambda (- y_2+ \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} (\gamma_1 + \gamma_2 y_1 ) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)\\
& + \mathcal{J}(y_1) \sum_{m=1}^M g_m \pi_d^m ( \phi_m(\bar{y}_1, y_2) - \Phi(y_1, y_2)) \\
& + \xi_p \mathcal{J}(y_1) \sum_{m=1}^M \pi_d^m \left(1 - g_m + g_m \log (g_m)\right)
\end{aligned}
$$

Or solve the following HJB with a terminal condition:

$$
\begin{aligned}
0 = \max_{\mathcal{E}}\min_{\omega_\ell }  & - \delta \Phi(y_1, y_2) +  \eta log(\mathcal{E}) \\
& + \frac{\partial \Phi}{\partial y_1} y_2 + \frac{\partial \Phi}{\partial y_2} \lambda (- y_2+ \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} (\gamma_1 + \gamma_2 y_1 ) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)
\end{aligned}
$$

$$
\Phi(\bar y_1, y_2) \approx  - \xi_p \log \left (\sum_{m=1}^M \pi_m^p \exp\left[-\frac{1}{\xi_p }\phi_m(\bar y_1, y_2) \right] \right) 
$$

In what follows, we show emission, $Y$ behavior before temperature anomay reaches the upper bound of jump threshold, $2^o C$, conditioning on no jump.

The $\lambda$ we choose here are:
- $\lambda = 0.116$, corresponds to half life of six years;
- $\lambda = 1, 2, 5$. Here, we try to illustrate emission behavior as $\lambda$ increase.

## Example 1: Half life of six years

In [65]:
λ = np.log(2) / 6
# parameters
δ = 0.01
η = 0.032
ξ_a = 0.01
θ_list = pd.read_csv("./data/model144.csv", header=None)[0].to_numpy()
θ_list = θ_list/1000
θ = np.mean(θ_list)
σy = 1.2*θ
# damage function
y_bar = 2.
γ_1 = 0.00017675
γ_2 = 2*0.0022
γ_3_list = np.linspace(0., 1./3, 20)
ξ_r = 1.
# %%
# y_grid
y1_step = .04
y1_grid = np.arange(0., 4. + y1_step, y1_step)

y2_step = .001
y2_grid = np.arange(0., .05 + y2_step, y2_step)
(y1_mat, y2_mat) = np.meshgrid(y1_grid,y2_grid, indexing = 'ij')

stateSpace = np.hstack([y1_mat.reshape(-1,1,order = 'F'),y2_mat.reshape(-1,1,order = 'F')])



In [66]:

args_list  = []
for γ_3_i in γ_3_list:
    args_iter = (y1_grid, y2_grid, γ_3_i, θ_list, (δ, η, γ_1, γ_2, y_bar, λ, ξ_a), 1e-6, 1., 1000, 0.05)
    args_list.append(args_iter)

if not os.path.exists(f"./data/res_list_{λ}"):
    with Pool() as p:
        res_list = p.starmap(solve_prep, args_list)

    with open(f"./data/res_list_{λ}", "wb") as handle:
        pickle.dump(res_list, handle)

# step II HJB:
with open(f"./data/res_list_{λ}", "rb") as file:
    res_list = pickle.load(file)

if not os.path.exists(f"./data/res_{λ}_{ξ_r}"):
    args_pre_jump = (δ, η, θ_list,  γ_1, γ_2, γ_3_list, ξ_a, ξ_r)
    res = solve_pre_jump_2state(res_list, args_pre_jump, ε=0.1)
    with open(f"./data/res_{λ}_{ξp}", "wb") as handle:
        pickle.dump(res, handle)

In [67]:
#load results
res = pickle.load(open(f"./data/res_{λ}_{ξ_r}", "rb"))

In [68]:
et_prejump, y1t_prejump, y2t_prejump = simulation_2d(res,
                                                  θ=np.mean(θ_list),
                                                  y1_0 = 1.1,
                                                  y2_0=np.mean(θ_list),
                                                  T=110
                                                 )

In [69]:
simul = {
    "et": et_prejump,
    "y1t": y1t_prejump,
    "y2t": y2t_prejump,
}

pickle.dump(simul, open(f"data/simul_{λ}", "wb"))

In [70]:
plot_2S_ey1y2(simul).write_html('plot_2S_ey1y2.html')
IFrame(src ='plot_2S_ey1y2.html', width=1090, height=500)


Compare the emission trajectories with the one state model:

In [41]:
et_1state = np.load("et_1state_cal.npy")
plot_1S_vs_2S_ems(et_1state, simul).write_html('plot_1S_vs_2S_ems.html')
IFrame(src ='plot_1S_vs_2S_ems.html', width=1090, height=500)


In [42]:
invkap = 0.09
α = 0.115
αₖ = - 0.043
σₖ = 0.0095
κ = 6.667
k0 = 85/α


In [43]:
Kt = simulate_logkapital(invkap, αₖ, σₖ, κ,  k0, T=111)
MC = δ*(1-η)/((α - invkap)*np.exp(Kt))
scc = η*(α - invkap)*np.exp(Kt)/(1-η)/et_prejump*1000
scc_1 = η*(α - invkap)*np.exp(Kt[:len(et_1state)])/(1-η)/et_1state*1000

In [44]:
# TODO: scc comparison
plot_1S_vs_2S_SCC(et_1state, scc, scc_1).write_html('plot_1S_vs_2S_SCC.html')
IFrame(src ='plot_1S_vs_2S_SCC.html', width=1090, height=500)


# Example 2: increasing $\lambda$

For the purpose of illustration, we choose $\lambda = 1, 2, 5$ to show what happens if the half life decreases.

In [80]:
res_dict = {}
λ_list = [1., 2., 5.]

for λ in λ_list:
    if not os.path.exists(f"./data/res_list_{λ}"):
        res_list = [delayed(solve_prep)(y1_grid, y2_grid, γ_3_i, θ_list, 
                                      (δ, η, γ_1, γ_2, y_bar, λ, ξ_a), 1e-6, 0.05, 10000, 0.05) for γ_3_i in γ_3_list]

        with open(f"./data/res_list_{λ}", "wb") as handle:
            pickle.dump(res_list, handle)

    # step II HJB:
    with open(f"./data/res_list_{λ}", "rb") as file:
        res_list = pickle.load(file)
    args_pre_jump = (δ, η, θ_list,  γ_1, γ_2, γ_3_list, ξ_a, ξ_r)
    res = solve_pre_jump_2state(res_list, args_pre_jump, ε=.1)

    with open(f"./data/res_{λ}_{ξ_r}", "wb") as handle:
        pickle.dump(res, handle)
    res_dict[λ] = res

Episode: 1	 lhs error: 0.590239450886	 rhs error: 0.582862279433
Episode: 2	 lhs error: 0.585747211780	 rhs error: 0.579987297957
Episode: 3	 lhs error: 0.573026568782	 rhs error: 0.568546689798
Episode: 4	 lhs error: 0.555821527020	 rhs error: 0.552346347325
Episode: 5	 lhs error: 0.535762269884	 rhs error: 0.533067451419
Episode: 6	 lhs error: 0.513799078689	 rhs error: 0.511702672447
Episode: 7	 lhs error: 0.490580834331	 rhs error: 0.488936261562
Episode: 8	 lhs error: 0.466592817455	 rhs error: 0.465283063589
Episode: 9	 lhs error: 0.442217376515	 rhs error: 0.441150116481
Episode: 10	 lhs error: 0.417764372348	 rhs error: 0.416867839722
Episode: 11	 lhs error: 0.393488089266	 rhs error: 0.392707540672
Episode: 12	 lhs error: 0.369597540743	 rhs error: 0.368892217097
Episode: 13	 lhs error: 0.346263358467	 rhs error: 0.345603866802
Episode: 14	 lhs error: 0.323622835739	 rhs error: 0.322988886512
Episode: 15	 lhs error: 0.301783962877	 rhs error: 0.301162443960
Episode: 16	 lhs er

Episode: 133	 lhs error: 0.041219858376	 rhs error: 0.121358672717
Episode: 134	 lhs error: 0.041078221809	 rhs error: 0.121275505233
Episode: 135	 lhs error: 0.040938701929	 rhs error: 0.121187338113
Episode: 136	 lhs error: 0.040798744864	 rhs error: 0.121094146183
Episode: 137	 lhs error: 0.040658357862	 rhs error: 0.120995905449
Episode: 138	 lhs error: 0.040517547886	 rhs error: 0.120892593106
Episode: 139	 lhs error: 0.040376321633	 rhs error: 0.120784187550
Episode: 140	 lhs error: 0.040234685538	 rhs error: 0.120670668388
Episode: 141	 lhs error: 0.040092645779	 rhs error: 0.120552016450
Episode: 142	 lhs error: 0.039950208294	 rhs error: 0.120428213799
Episode: 143	 lhs error: 0.039807378787	 rhs error: 0.120299243743
Episode: 144	 lhs error: 0.039664162738	 rhs error: 0.120165090846
Episode: 145	 lhs error: 0.039520565405	 rhs error: 0.120025740936
Episode: 146	 lhs error: 0.039376591839	 rhs error: 0.119881181118
Episode: 147	 lhs error: 0.039232246887	 rhs error: 0.11973139

Episode: 256	 lhs error: 0.028635579498	 rhs error: 0.118559530497
Episode: 257	 lhs error: 0.028549575843	 rhs error: 0.118252600323
Episode: 258	 lhs error: 0.028462951845	 rhs error: 0.117941080707
Episode: 259	 lhs error: 0.028375709199	 rhs error: 0.117625026917
Episode: 260	 lhs error: 0.028287849631	 rhs error: 0.117304494918
Episode: 261	 lhs error: 0.028199374914	 rhs error: 0.116979541348
Episode: 262	 lhs error: 0.028110286867	 rhs error: 0.116650223508
Episode: 263	 lhs error: 0.028020587362	 rhs error: 0.116316599350
Episode: 264	 lhs error: 0.027930278328	 rhs error: 0.115978727471
Episode: 265	 lhs error: 0.027839361760	 rhs error: 0.115636667104
Episode: 266	 lhs error: 0.027747839716	 rhs error: 0.115290478119
Episode: 267	 lhs error: 0.027655714328	 rhs error: 0.114940221020
Episode: 268	 lhs error: 0.027562987800	 rhs error: 0.114585956940
Episode: 269	 lhs error: 0.027469662416	 rhs error: 0.114227747643
Episode: 270	 lhs error: 0.027375740539	 rhs error: 0.11386565

Episode: 384	 lhs error: 0.021867797770	 rhs error: 0.075535232005
Episode: 385	 lhs error: 0.021833027521	 rhs error: 0.075226210589
Episode: 386	 lhs error: 0.021797813397	 rhs error: 0.074915450772
Episode: 387	 lhs error: 0.021762155575	 rhs error: 0.074603000530
Episode: 388	 lhs error: 0.021726054246	 rhs error: 0.074288907775
Episode: 389	 lhs error: 0.021689509623	 rhs error: 0.073973220360
Episode: 390	 lhs error: 0.021652521934	 rhs error: 0.073655986086
Episode: 391	 lhs error: 0.021615091429	 rhs error: 0.073337252700
Episode: 392	 lhs error: 0.021577218376	 rhs error: 0.073017067902
Episode: 393	 lhs error: 0.021538903067	 rhs error: 0.072695479343
Episode: 394	 lhs error: 0.021500145814	 rhs error: 0.072372534624
Episode: 395	 lhs error: 0.021460946951	 rhs error: 0.072048281292
Episode: 396	 lhs error: 0.021421306839	 rhs error: 0.071722766838
Episode: 397	 lhs error: 0.021381225860	 rhs error: 0.071396038688
Episode: 398	 lhs error: 0.021340704423	 rhs error: 0.07106814

Episode: 512	 lhs error: 0.017384449757	 rhs error: 0.038948972930
Episode: 513	 lhs error: 0.017360949682	 rhs error: 0.038716277874
Episode: 514	 lhs error: 0.017337148838	 rhs error: 0.038483369331
Episode: 515	 lhs error: 0.017313047148	 rhs error: 0.038250275895
Episode: 516	 lhs error: 0.017288644546	 rhs error: 0.038017025906
Episode: 517	 lhs error: 0.017263940983	 rhs error: 0.037783647447
Episode: 518	 lhs error: 0.017238936422	 rhs error: 0.037550168335
Episode: 519	 lhs error: 0.017213630842	 rhs error: 0.037316616123
Episode: 520	 lhs error: 0.017188024233	 rhs error: 0.037083018085
Episode: 521	 lhs error: 0.017162116604	 rhs error: 0.036849401220
Episode: 522	 lhs error: 0.017135907973	 rhs error: 0.036615792239
Episode: 523	 lhs error: 0.017109398377	 rhs error: 0.036382217563
Episode: 524	 lhs error: 0.017082587865	 rhs error: 0.036148703314
Episode: 525	 lhs error: 0.017055474172	 rhs error: 0.035915275308
Episode: 526	 lhs error: 0.017028061976	 rhs error: 0.03568195

...

Episode: 2093	 lhs error: 0.000539300330	 rhs error: 0.035180153955
Episode: 2094	 lhs error: 0.000534829409	 rhs error: 0.035180153955
Episode: 2095	 lhs error: 0.000530388550	 rhs error: 0.035180153955
Episode: 2096	 lhs error: 0.000525977631	 rhs error: 0.035180153955
Episode: 2097	 lhs error: 0.000521596526	 rhs error: 0.035180153955
Episode: 2098	 lhs error: 0.000517245112	 rhs error: 0.035180153955
Episode: 2099	 lhs error: 0.000512923270	 rhs error: 0.035180153955
Episode: 2100	 lhs error: 0.000508630883	 rhs error: 0.035180153955
Episode: 2101	 lhs error: 0.000504367838	 rhs error: 0.035180153955
Episode: 2102	 lhs error: 0.000500134020	 rhs error: 0.035180153955
Episode: 2103	 lhs error: 0.000495929314	 rhs error: 0.035180153955
Episode: 2104	 lhs error: 0.000491753128	 rhs error: 0.035180153955
Episode: 2105	 lhs error: 0.000487606439	 rhs error: 0.035180153955
Episode: 2106	 lhs error: 0.000483488435	 rhs error: 0.035180153955
Episode: 2107	 lhs error: 0.000479399118	 rhs er

Episode: 2214	 lhs error: 0.000180882188	 rhs error: 0.035180153955
Episode: 2215	 lhs error: 0.000179143585	 rhs error: 0.035180153955
Episode: 2216	 lhs error: 0.000177419996	 rhs error: 0.035180153955
Episode: 2217	 lhs error: 0.000175711380	 rhs error: 0.035180153955
Episode: 2218	 lhs error: 0.000174017682	 rhs error: 0.035180153955
Episode: 2219	 lhs error: 0.000172338821	 rhs error: 0.035180153955
Episode: 2220	 lhs error: 0.000170674694	 rhs error: 0.035180153955
Episode: 2221	 lhs error: 0.000169025185	 rhs error: 0.035180153955
Episode: 2222	 lhs error: 0.000167390175	 rhs error: 0.035180153955
Episode: 2223	 lhs error: 0.000165769545	 rhs error: 0.035180153955
Episode: 2224	 lhs error: 0.000164163178	 rhs error: 0.035180153955
Episode: 2225	 lhs error: 0.000162570960	 rhs error: 0.035180153955
Episode: 2226	 lhs error: 0.000160992782	 rhs error: 0.035180153955
Episode: 2227	 lhs error: 0.000159428534	 rhs error: 0.035180153955
Episode: 2228	 lhs error: 0.000157878110	 rhs er

Episode: 2343	 lhs error: 0.000048677950	 rhs error: 0.035180153955
Episode: 2344	 lhs error: 0.000048162398	 rhs error: 0.035180153955
Episode: 2345	 lhs error: 0.000047652539	 rhs error: 0.035180153955
Episode: 2346	 lhs error: 0.000047147207	 rhs error: 0.035180153955
Episode: 2347	 lhs error: 0.000046646940	 rhs error: 0.035180153955
Episode: 2348	 lhs error: 0.000046151609	 rhs error: 0.035180153955
Episode: 2349	 lhs error: 0.000045661147	 rhs error: 0.035180153955
Episode: 2350	 lhs error: 0.000045175585	 rhs error: 0.035180153955
Episode: 2351	 lhs error: 0.000044694959	 rhs error: 0.035180153955
Episode: 2352	 lhs error: 0.000044219264	 rhs error: 0.035180153955
Episode: 2353	 lhs error: 0.000043748454	 rhs error: 0.035180153955
Episode: 2354	 lhs error: 0.000043282460	 rhs error: 0.035180153955
Episode: 2355	 lhs error: 0.000042821206	 rhs error: 0.035180153955
Episode: 2356	 lhs error: 0.000042364622	 rhs error: 0.035180153955
Episode: 2357	 lhs error: 0.000041912646	 rhs er

Episode: 2469	 lhs error: 0.000012125254	 rhs error: 0.035180153955
Episode: 2470	 lhs error: 0.000011987859	 rhs error: 0.035180153955
Episode: 2471	 lhs error: 0.000011852038	 rhs error: 0.035180153955
Episode: 2472	 lhs error: 0.000011717746	 rhs error: 0.035180153955
Episode: 2473	 lhs error: 0.000011584946	 rhs error: 0.035180153955
Episode: 2474	 lhs error: 0.000011453607	 rhs error: 0.035180153955
Episode: 2475	 lhs error: 0.000011323705	 rhs error: 0.035180153955
Episode: 2476	 lhs error: 0.000011195221	 rhs error: 0.035180153955
Episode: 2477	 lhs error: 0.000011068135	 rhs error: 0.035180153955
Episode: 2478	 lhs error: 0.000010942432	 rhs error: 0.035180153955
Episode: 2479	 lhs error: 0.000010818096	 rhs error: 0.035180153955
Episode: 2480	 lhs error: 0.000010694743	 rhs error: 0.035180153955
Episode: 2481	 lhs error: 0.000010573095	 rhs error: 0.035180153955
Episode: 2482	 lhs error: 0.000010452768	 rhs error: 0.035180153955
Episode: 2483	 lhs error: 0.000010333765	 rhs er

Episode: 2590	 lhs error: 0.000002949926	 rhs error: 0.035180153955
Episode: 2591	 lhs error: 0.000002914346	 rhs error: 0.035180153955
Episode: 2592	 lhs error: 0.000002879213	 rhs error: 0.035180153955
Episode: 2593	 lhs error: 0.000002844543	 rhs error: 0.035180153955
Episode: 2594	 lhs error: 0.000002810011	 rhs error: 0.035180153955
Episode: 2595	 lhs error: 0.000002776197	 rhs error: 0.035180153955
Episode: 2596	 lhs error: 0.000002743179	 rhs error: 0.035180153955
Episode: 2597	 lhs error: 0.000002710667	 rhs error: 0.035180153955
Episode: 2598	 lhs error: 0.000002678462	 rhs error: 0.035180153955
Episode: 2599	 lhs error: 0.000002646506	 rhs error: 0.035180153955
Episode: 2600	 lhs error: 0.000002614817	 rhs error: 0.035180153955
Episode: 2601	 lhs error: 0.000002583437	 rhs error: 0.035180153955
Episode: 2602	 lhs error: 0.000002552401	 rhs error: 0.035180153955
Episode: 2603	 lhs error: 0.000002521728	 rhs error: 0.035180153955
Episode: 2604	 lhs error: 0.000002491426	 rhs er

In [81]:
simul_list = {}
for λ in λ_list:
    res_temp=pickle.load(open(f"./data/res_{λ}_{ξ_r}", "rb"))
    et_prejump, y1t_prejump, y2t_prejump = simulation_2d(res_temp,
                                                  θ=np.mean(θ_list),
                                                  y1_0 = 1.1,
                                                  y2_0=np.mean(θ_list),
                                                  T=110
                                                 )
    simul = {
    "et": et_prejump,
    "y1t": y1t_prejump,
    "y2t": y2t_prejump,
    }
    simul_list[λ] = simul

pickle.dump(simul, open(f"data/simul_{λ}", "wb"))

In [82]:
# plot, emission, y1, y2, button, λ = 1, 2, 5
plot_2S_ey1y2_multi_λ(simul_list, λ_list).write_html('plot_2S_ey1y2_multi_λ.html')
IFrame(src ='plot_2S_ey1y2_multi_λ.html', width=1090, height=500)


In [83]:
#  emission, λ = 1,2,5 and 1 state result
plot_1S_vs_2S_ems_multi_λ(et_1state, simul_list, λ_list).write_html('plot_1S_vs_2S_ems_multi_λ.html')
IFrame(src ='plot_1S_vs_2S_ems_multi_λ.html', width=1090, height=500)


In [77]:
scc_list=np.zeros((3, 111))
for i in range(len(λ_list)):
    scc_list[i] = η*(α - invkap)*np.exp(Kt)/(1-η)/simul_list[λ_list[i]]["et"]*1000

In [78]:
# scc λ = 1, 2, 5 and 1 state

plot_1S_vs_2S_SCC_multi_λ(et_1state, scc_list, scc_1, λ_list).write_html('plot_1S_vs_2S_SCC_multi_λ.html')
IFrame(src ='plot_1S_vs_2S_SCC_multi_λ.html', width=1090, height=500)